In [1]:
import pandas as pd
import numpy as np
import re
import os

In [2]:
''' 
foodexp.csv : Details, period (null items)
columns : type of food, item name, period, details, storage'''

' \nfoodexp.csv : Details, period (null items)\ncolumns : type of food, item name, period, details, storage'

In [3]:
def days_conversion(x):
    if "day" in x:
        x = x.split(' ')[0]
        if len(x)==1:
            return int(x)
        else:
            return int(x[x.find('-')+1])
    elif "week" in x:
        x = x.split(' ')[0]
        if len(x)==1:
            return int(x)*7
        else:
            return int(x[x.find('-')+1])*7
    elif "month" in x:
        x = x.split(' ')[0]
        if len(x)==1:
            return int(x)*30
        else:
            return int(x[x.find('-')+1])*30
    elif "year" in x:
        x = x.split(' ')[0]
        if len(x)==1:
            return int(x)*365
        else:
            return int(x[x.find('-')+1])*365
    elif "hour" in x:
        x = x.split(' ')[0]
        if len(x)==1:
            return int(x)/24
        else:
            return int(x[x.find('-')+1])/24
    elif "date" in x and re.match('^[0-9]', x):
        if 'mo' in x:
            return int(x.split(' ')[0])*30
        if 'week' in x:
            return int(x.split(' ')[0])*7
        if 'year' in x:
            return int(x.split(' ')[0])*365
        else:
            return int(x.split(' ')[0])
    else:
        return np.nan
    
        

In [11]:
combined_csv.shape

(1358, 6)

In [12]:
def period_conversion(df):
    non_null_df = df[df['PERIOD'].notnull()]
    null_df = df[df['PERIOD'].isnull()]
    # Change period column to days and consider the maximum value
    non_null_df['PERIOD'] = non_null_df['PERIOD'].apply(days_conversion)
    df = pd.concat([non_null_df,null_df],ignore_index=True)
    df["PERIOD"] = pd.to_numeric(df["PERIOD"])
    # fill period column null values by "TYPE" and "STORAGE"
    df['PERIOD'] = df.groupby(['TYPE'])['PERIOD'].transform(lambda x:x.fillna(round(np.mean(x),2)))
    return df

In [13]:
csv_files = [f for f in os.listdir() if f.endswith('.csv') and f.startswith('food')]
# 1. Merge all food.csv files
combined_csv = pd.concat([pd.read_csv(f) for f in csv_files])
# removing redundant items (by eliminating nan records)
combined_csv = combined_csv.drop_duplicates()
# separating null and non-null rows to two dataframes
combined_csv = period_conversion(combined_csv)
# Split item name and create new column
combined_csv[['ITEM','ITEM_DESC']] = combined_csv.ITEM.str.split(',', 1, expand=True)

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [14]:
combined_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679 entries, 0 to 678
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TYPE       679 non-null    object 
 1   ITEM       679 non-null    object 
 2   PERIOD     679 non-null    float64
 3   DETAILS    307 non-null    object 
 4   STORAGE    679 non-null    object 
 5   ITEM_DESC  455 non-null    object 
dtypes: float64(1), object(5)
memory usage: 32.0+ KB


In [15]:
orderHeaderDonation_df = pd.read_csv("orderHeaderDonations.csv")
orderHeaderRequest_df = pd.read_csv("orderHeaderRequests.csv")

In [16]:
orderHeaderDonation_df['OrderDate'] = pd.to_datetime(orderHeaderDonation_df['OrderDate'], format='%Y-%m-%d')
orderHeaderRequest_df['OrderDate'] = pd.to_datetime(orderHeaderRequest_df ['OrderDate'], format='%Y-%m-%d')

In [17]:
orderHeaderDonation_df.OrderId = orderHeaderDonation_df.OrderId.astype(str)
orderHeaderDonation_df.PartnerId = orderHeaderDonation_df.PartnerId.astype(str)

In [18]:
orderHeaderRequest_df.OrderId = orderHeaderRequest_df.OrderId.astype(str)
orderHeaderRequest_df.CustomerId = orderHeaderRequest_df.CustomerId.astype(str)

In [19]:
orderHeaderRequest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   OrderId         392 non-null    object        
 1   CustomerId      392 non-null    object        
 2   OrderTotal      392 non-null    int64         
 3   OrderDate       392 non-null    datetime64[ns]
 4   Location        392 non-null    object        
 5   CustomerName    392 non-null    object        
 6   CustomerGeoLat  392 non-null    float64       
 7   CustomerGeoLon  392 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 24.6+ KB


In [20]:
orderItemsDonation_df = pd.read_csv("orderItemsDonations.csv")
orderItemRequest_df = pd.read_csv("orderItemsRequests.csv")

In [21]:
orderItemsDonation_df.OrderId = orderItemsDonation_df.OrderId.astype(str)
orderItemsDonation_df.OrderItemId = orderItemsDonation_df.OrderItemId.astype(str) 
orderItemRequest_df.OrderId = orderItemRequest_df.OrderId.astype(str) 
orderItemRequest_df.OrderItemId = orderItemRequest_df.OrderItemId.astype(str) 

In [22]:
orderItemsDonation_df.columns = ['OrderId','OrderItemId','TYPE','ITEM','PERIOD','DETAILS','STORAGE','Quantity']
orderItemRequest_df.columns = ['OrderId','OrderItemId','TYPE','ITEM','PERIOD','DETAILS','STORAGE']

In [23]:
orderItemsDonation_df.head()

,OrderId,OrderItemId,TYPE,ITEM,PERIOD,DETAILS,STORAGE,Quantity
0,20051,1,DIARY,"Cheese, processed slices",1-2 months,NaN,REFRIGERATED,105
1,20051,2,DIARY,"Eggs, in shell",3-5 weeks,NaN,REFRIGERATED,120
2,20051,3,DIARY,Margarine,12 months,NaN,FROZEN,140
3,20051,4,DIARY,"Milk, plain or flavored",1 week,NaN,REFRIGERATED,135
4,20051,5,SOY,Tofu,1 week,NaN,REFRIGERATED,145


In [24]:
orderItemsDonation_df[['ITEM','ITEM_DESC']] = orderItemsDonation_df.ITEM.str.split(',', 1, expand=True)
orderItemRequest_df[['ITEM','ITEM_DESC']] = orderItemRequest_df.ITEM.str.split(',', 1, expand=True)

In [25]:
orderItemsDonation_df = period_conversion(orderItemsDonation_df)
orderItemRequest_df = period_conversion(orderItemRequest_df)

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [26]:
orderItemsDonation_df.head()

,OrderId,OrderItemId,TYPE,ITEM,PERIOD,DETAILS,STORAGE,Quantity,ITEM_DESC
0,20051,1,DIARY,Cheese,60.0,NaN,REFRIGERATED,105,processed slices
1,20051,2,DIARY,Eggs,35.0,NaN,REFRIGERATED,120,in shell
2,20051,3,DIARY,Margarine,30.0,NaN,FROZEN,140,None
3,20051,4,DIARY,Milk,7.0,NaN,REFRIGERATED,135,plain or flavored
4,20051,5,SOY,Tofu,7.0,NaN,REFRIGERATED,145,None


In [27]:
combined_csv.to_csv("food_data_cleaned.csv", index=False)
orderHeaderRequest_df.to_csv('orderHeaderRequests_cleaned.csv',index=False)
orderHeaderDonation_df.to_csv('orderHeaderDonations_cleaned.csv',index=False)
orderItemsDonation_df.to_csv("orderItemsDonation_cleaned.csv", index=False)
orderItemRequest_df.to_csv("orderItemRequest_cleaned.csv", index=False)